# Notebook to recenter images of a DICOM dataset
Use this notebook to navigate and select a location in the image to recenter a series of images. It loads an specific series and uses the notebook to select a location.

This notebook shows an example how to process a CT dataset. Some tricks need to be done to be sure it can be loaded into the Sonalleve treatment planning. 

**Very important, make as many copies of this notebook for each transformation applied. Like that the notebook is also the documentation of the transformation applied. This will be extremely useful for future analysis or for people reviewing the process**

** Also important. Operations are accumulative, meaning that you have to be careful of applying only once the desired translation/rotation.** If you need to start from scratch, simply re-execute from the cell with the code `A=SelectionMarkersUI(...`

Do not forget to execute cells with "shift-Enter"


In [1]:
%matplotlib nbagg

In [2]:
from __future__ import print_function
from RecenterDicom import *
import matplotlib.pyplot as plt

base_dir = '/Users/avadnly/Desktop/ReCenter_DICOMs/sorted/Patient_Pat_fetal_HIFU_4/AX HASTE MATERNAL'

## Select a directory that contains the DICOMDIR file of the exported dataset
Assign the value to the  ``base_dir`` variable. The code will print a list of all the series. Be sure of having inspected the dataset before with Osirix or any other DICOM viewer so you have an idea already of which dataset to select

## Select a CT scan series by their UID
Just copy the text of the UID above of the series you want to process. This is the best way to be sure you are loading the correct dataset.

#### OPTIONAL: Apply a rotation
Many datasets are not necessarily  showing a patient in the right orientation, especially for furhter analysis with the Sonalleve SW. If the dataset is required to be rotated, **DO IT BEFORE THE RECENTERING!!!!**.
Uncomment and execute the code below if you need a rotation.

#### Note. The axis selection seems to be kind of trial and error. 
For some datasets it rotates correctly, but for some other it rotates in one of other axis. Still need to figure out. But a little try and error for the moment will be still required to rotate the image as needed. Try a combination with axis either being 'RL', 'HF' or 'AP'

In [3]:
A=SelectionMarkersUI(base_dir, False, DICOMDIR=False, ROT=2)
A.Rotation(90,axis='LR')

## Use the interactive tool to select the number of slice, and (x,y)-coordinates in pixel of the location you want to use as the place to recenter the dataset.
By default the 3D visualization is off to help to select faster the location. Once the location is identified, activating the 3D visualization will help to confim its location in the MRI coordinate system. At this moment, the recenter hasn't yet been applied in the images. 

If already know the location of slice, x-coordinate and y-coordinate (which may the case when reprocessing), you can comment the call to `InteractiveSel()` and un-comment the `ShowData` function call where you specify the # slice and x- and y-coordinates directly. This may be handy to keep documented the changes.

In [4]:
A.InteractiveSel()
#A.ShowData(0.5,67,191,338)


aW50ZXJhY3RpdmUoY2hpbGRyZW49KEZsb2F0U2xpZGVyKHZhbHVlPTAuNSwgY29udGludW91c191cGRhdGU9RmFsc2UsIGRlc2NyaXB0aW9uPXUnVG9wQ29udHJhc3QnLCBtYXg9MS4wLCBtaW7igKY=


##  "Execute" the offset to the dataset
The selected location during the navigation is now applied with the code below

In [5]:
A.ApplyOffset()


##  Export the data
Specify a directory where to export the data. If the directory does not exist, it will be created. Be sure of avoiding directories with original data. Also, if you call again this step later, all images in the specified subdirectory will be overwritten.

You can specify the flag 'bDeleteFilesFirst=True' to delete files in the target directory. This is useful if you are reusing the same directory over and over. If you end exporting a dataset with less images than a previous exports,it may cause problems in the Sonalleve.

In [6]:
A.ExportData('../ExportedForSonnalleve',bDeleteFilesFirst=False)


##  Use the data
The exported data is conformal with DICOM viewers and the Sonalleve software. For the Sonalleve, start an standalone operation, input a body temperature. 

Modify the code below to match your Sonalleve directory, the location of dcmtk and the directory where you exported the data. Just run it with shift-Enter as usual and the images should be imported in the Sonalleve. Be sure the last line is correct to put back the current working directory in the notebook.

```
import os
curdir=os.getcwd()
%cd "C:/Program Files (x86)/Philips Medical Systems/HIFU/3.2.740.2311"
%run ./Scripts/DICOM/PlanningImageImporter.py -d "C:\Patient Data\ExportedForSonnalleve" -t "C:\dcmtk-3.6.0-win32-i386\bin"
# DO NOT FORGET TO PUT BACK YOUR CURRENT DIRECTORY
%cd "C:/Users/Charles/Desktop/ReCenterMRI"
```